In [122]:
import pickle
import numpy as np
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))  

In [123]:
with open("/home/jackragless/Downloads/wiki_master_dataframe_chinked_wgrammar.pkl", 'rb') as f:
    corpus = pickle.load(f)

In [124]:
kw_pool = []
for page in corpus:
    kw_pool += page['kw']
kw_pool = np.array(list(set(kw_pool)))
kw_pool = np.sort(kw_pool)

In [125]:
def para_kw_space_remove(curpage, kw_pool):
    curstring = curpage['text']
    for kw in kw_pool:
        if curstring.find(kw)!=-1:
            highlight = curstring[curstring.find(kw):curstring.find(kw)+len(kw)].replace(' ', '|/|')
            curstring = str(curstring[:curstring.find(kw)]) + str(highlight) + str(curstring[curstring.find(kw)+len(kw):])
    return curstring

In [126]:
def biogen(curpage, kw_pool):
    sents = nltk.sent_tokenize(para_kw_space_remove(curpage, kw_pool))
    final_label_arr = []
    for sent in sents:
        temp_label_arr = []
        if len(sent)<=512:
            for phrase in nltk.word_tokenize(sent):
                count = 0
                if phrase.replace('|/|',' ') in kw_pool:
                    phrase_words = phrase.replace('|/|',' ').split()
                    for word in phrase_words:
                        if count == 0:
                            temp_label_arr.append('B')
                        elif count > 0:
                            temp_label_arr.append('I')
                        count+=1
                else:
                    temp_label_arr.append('O')
        final_label_arr.append(temp_label_arr)

    final_sent_arr = []
    for sent in sents:
        final_sent_arr.append(nltk.word_tokenize(sent.replace('|/|',' ')))

    return final_sent_arr, final_label_arr

In [127]:
def same_len(sentences, labels):
    filt_s = []
    filt_l = []
    for i in range(len(sentences)):
        if len(sentences[i]) == len(labels[i]):
            filt_s.append(sentences[i])
            filt_l.append(labels[i])
    return filt_s, filt_l

In [128]:
labels = []
sentences = []
count = 0
for page in corpus:
    count += 1
    print(str(count) + '/' + str(len(corpus)), end='\r')
    s, l= biogen(page, kw_pool)
    sentences += s
    labels += l

KeyboardInterrupt: 

In [ ]:
sentences, labels = same_len(sentences, labels)

In [ ]:
final_sent = []
final_label = []
for sent in range(len(sentences)):
    temp_sent = []
    temp_label = []
    for word in range(len(sentences[sent])):
#         if sentences[sent][word].lower() not in stop_words:
        temp_sent.append(sentences[sent][word])
        temp_label.append(labels[sent][word])
    final_sent.append(temp_sent)
    final_label.append(temp_label)
FINAL = [final_sent, final_label]

In [ ]:
# for i in range(len(FINAL[0])):
#     print(FINAL[0][i], '\n', FINAL[1][i])

In [ ]:
fileObj = open('../../data/biogen_stopwords_latest_wgrammar.pkl', 'wb')
pickle.dump(FINAL,fileObj)

In [ ]:
# for i in corpus:
#     if i['title'] == 'Intel AMT versions':
#         print(i['text'])

In [ ]:
# sentences_tok = []
# count= 0 
# for sent in sentences:
#     count += 1
#     print(count, end='\r')
#     sent = sent.replace('|/|',' ')
#     sentences_tok.append(nltk.word_tokenize(sent)[:-1])

In [ ]:
# BIO_df = []
# sent_count = 0
# fail_count = 0
# for i in range(len(sentences_tok)):
#     sent_count += 1
#     if len(labels[i]) > 0 and len(labels[i]) == len(sentences_tok[i]):
#         for j in range(len(sentences_tok[i])):
#             BIO_df.append([sent_count, sentences_tok[i][j], labels[i][j]])
#     else:
#         fail_count += 1

In [ ]:
# fileObj = open('biogen_stopwords_incl', 'wb')
# pickle.dump(BIO_df,fileObj)